In [1]:
from dotenv import load_dotenv
import os
from pathlib import Path
import requests
from requests import Request, Session
import datetime
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import time
from kucoin.trade.trade import TradeData
from kucoin.client import Trade, User, Market

In [2]:
load_dotenv();
api_key = os.getenv('KUCOIN_API_KEY')
api_secret = os.getenv('KUCOIN_SECRET_KEY')
api_passphrase = os.getenv('KUCOIN_PASSPHRASE')

In [3]:
Tclient = Trade(api_key, api_secret, api_passphrase)
Uclient = User(api_key, api_secret, api_passphrase)
Mclient = Market(api_key, api_secret, api_passphrase)

### Define a function that gets latest best bid data from KuCoin orderbook

In [4]:
def kucoin_func(ticker):
    # Request data from api
    crypto = requests.get(f'https://api.kucoin.com/api/v1/market/orderbook/level1?symbol={ticker}-USDT')
    
    # Import api data as json    
    crypto_j = crypto.json()
    
    # Create pandas df from json file    
    crypto_df = pd.DataFrame(crypto_j['data'], index=[0])
    
    # Rename columns
    crypto_df = pd.DataFrame(crypto_df['bestBid']).rename(columns = {'bestBid':ticker})
    
    #Transpose df
    crypto_df = crypto_df.T
    
    #Rename Column Again *There's probably an easier way to do this*
    crypto_df.rename(columns={0:'bid'}, inplace=True)
    
    return crypto_df

#### Get account balances

In [5]:
account = pd.DataFrame(Uclient.get_account_list())
account = account.loc[account['type']=='trade']
account = account.loc[account['balance']!='0']
account.drop(columns=['id','holds'],inplace=True)
account.set_index('currency', inplace=True)
non_lev = []
for token in account.index:
    if token[-2] != '3':
        non_lev.append(token)
account = account.drop(non_lev,axis=0)
account.sort_index(inplace=True)
account.drop(columns='type',inplace=True)

#### Begin creating central dataframe
##### This first step returns leveraged tokens in account and number of tokens

In [6]:
lev_token = pd.read_csv('lev_token.csv', index_col=0)

In [7]:
difference = list(set(account.index)-set(lev_token.index))
for token in difference:
    lev_token = lev_token.append(account.loc[token])

In [8]:
lev_token

,balance,available,bid,price,dollarvalue
FTM3L,32.363728,32.363728,0.357200,0.309609,11.560324
FTM3S,47.513364,47.513364,0.173000,0.210890,8.219812
SAND3L,3.810719,3.810719,3.340800,2.629454,12.730849
SAND3S,159.676006,159.676006,0.045441,0.062753,7.255837
SUSHI3L,198.693918,198.693918,0.052127,0.043625,10.357318
SUSHI3S,0.373935,0.373935,22.973300,28.952224,8.590520
NEAR3L,7.411046,7.411046,1.664900,1.352051,12.338651
NEAR3S,2518.897836,2518.897836,0.002696,0.003978,6.790949
MATIC3S,22.880229,22.880229,0.388800,0.437938,8.895833
MATIC3L,8.865557,8.865557,1.206300,1.130229,10.694521


In [9]:
for token in account.index:
                         
    if account.loc[token]['balance'] != lev_token.loc[token]['balance']:
        lev_token.loc[token]['balance'] = account.loc[token]['balance']
    else:
        pass
                                                     
    if account.loc[token]['available'] != lev_token.loc[token]['available']:
        lev_token.loc[token]['available'] = account.loc[token]['available']
    else:
        pass
    

In [10]:
lev_token

,balance,available,bid,price,dollarvalue
FTM3L,32.363728,32.363728,0.357200,0.309609,11.560324
FTM3S,47.513364,47.513364,0.173000,0.210890,8.219812
SAND3L,3.810719,3.810719,3.340800,2.629454,12.730849
SAND3S,159.676006,159.676006,0.045441,0.062753,7.255837
SUSHI3L,198.693918,198.693918,0.052127,0.043625,10.357318
SUSHI3S,0.373935,0.373935,22.973300,28.952224,8.590520
NEAR3L,7.411046,7.411046,1.664900,1.352051,12.338651
NEAR3S,2518.897836,2518.897836,0.002696,0.003978,6.790949
MATIC3S,22.880229,22.880229,0.388800,0.437938,8.895833
MATIC3L,8.865557,8.865557,1.206300,1.130229,10.694521


#### Append latest bid data for each token using function defined above

In [11]:
bid_df=pd.DataFrame()
for token in lev_token.index:
    bid_df = bid_df.append(kucoin_func(token))
    lev_token.loc[token]['bid'] = bid_df.loc[token]['bid']

#### Create a 'dollar value' column using latest bid price and available balance

In [12]:
lev_token = lev_token[['balance','available','bid','price']].astype(float)
lev_token['dollarvalue']= lev_token['available']*lev_token['bid']

In [13]:
lev_token

,balance,available,bid,price,dollarvalue
FTM3L,32.363728,32.363728,0.407000,0.309609,13.172037
FTM3S,47.513364,47.513364,0.149000,0.210890,7.079491
SAND3L,3.810719,3.810719,4.512900,2.629454,17.197392
SAND3S,159.676006,159.676006,0.029702,0.062753,4.742697
SUSHI3L,198.693918,198.693918,0.072101,0.043625,14.326030
SUSHI3S,0.373935,0.373935,14.580900,28.952224,5.452308
NEAR3L,7.411046,7.411046,1.931800,1.352051,14.316659
NEAR3S,2518.897836,2518.897836,0.002266,0.003978,5.707822
MATIC3S,22.880229,22.880229,0.335000,0.437938,7.664877
MATIC3L,8.865557,8.865557,1.376900,1.130229,12.206985


#### Retrieve price paid for token from recent orders

In [14]:
orders = pd.DataFrame(Tclient.get_recent_orders())

orders = orders.loc[orders['side']=='buy']

orders['price'] = orders['funds'].astype(float)*1.001/(orders['dealSize']).astype(float)
orders.reset_index(inplace=True, drop=True)
s=[]
for x in range(len(orders.index)):
    g = orders['symbol'][x]
    f = g[:-5]
    s.append(f)
orders['ticker'] = s
# fills.set_index('ticker',drop=True,inplace=True)

In [15]:
orders

,id,symbol,opType,type,side,price,size,funds,dealFunds,dealSize,...,cancelAfter,channel,clientOid,remark,tags,isActive,cancelExist,createdAt,tradeType,ticker
0,61fd50ca352643000139e03f,MATIC3S-USDT,DEAL,market,buy,0.437938,0,10.0101,10.010099999375,22.88022857,...,0,ANDROID,None,None,None,False,False,1643991242966,TRADE,MATIC3S
1,61fd50beb6f11000015eb3df,MATIC3L-USDT,DEAL,market,buy,1.130229,0,10.0101,10.010099990933,8.86555663,...,0,ANDROID,None,None,None,False,False,1643991230443,TRADE,MATIC3L


#### Update dataframe with price paid info for each token

In [16]:
ticker_price=pd.read_csv('ticker_price.csv',index_col=0)
difference = list(set(account.index)-set(ticker_price.index))
for token in difference:
    ticker_price = ticker_price.append(account.loc[token])
    ticker_price = ticker_price.drop(columns=['available','balance'])
ticker_price = ticker_price.fillna(0)

In [17]:
for ticker in ticker_price.index:
    if ticker_price.loc[ticker]['price'] == 0:
        ticker_price.loc[ticker]['price'] = orders['price'].loc[orders['ticker']==ticker]

In [18]:
ticker_price

,price
ticker,
FTM3L,0.309609
FTM3S,0.210890
SAND3L,2.629454
SAND3S,0.062753
SUSHI3L,0.043625
SUSHI3S,28.952224
NEAR3L,1.352051
NEAR3S,0.003978
MATIC3S,0.437938


In [19]:
for ticker in lev_token.index:
    d = orders.loc[orders['ticker']==ticker]
    d = d.sort_values('createdAt', ascending = False)
    d = d.loc[d['createdAt'] == d['createdAt'].max()]
    price = pd.DataFrame(d.price.values,index=d.ticker).rename(columns={0:'pricePaid'})
    if ticker_price.loc[ticker]['price'] != lev_token.loc[ticker]['price']:
        lev_token.loc[ticker]['price'] = ticker_price.loc[ticker]['price']
    else:
        pass

In [20]:
ticker_price.to_csv('ticker_price.csv')

In [21]:
lev_token

,balance,available,bid,price,dollarvalue
FTM3L,32.363728,32.363728,0.407000,0.309609,13.172037
FTM3S,47.513364,47.513364,0.149000,0.210890,7.079491
SAND3L,3.810719,3.810719,4.512900,2.629454,17.197392
SAND3S,159.676006,159.676006,0.029702,0.062753,4.742697
SUSHI3L,198.693918,198.693918,0.072101,0.043625,14.326030
SUSHI3S,0.373935,0.373935,14.580900,28.952224,5.452308
NEAR3L,7.411046,7.411046,1.931800,1.352051,14.316659
NEAR3S,2518.897836,2518.897836,0.002266,0.003978,5.707822
MATIC3S,22.880229,22.880229,0.335000,0.437938,7.664877
MATIC3L,8.865557,8.865557,1.376900,1.130229,12.206985


In [22]:
lev_token.to_csv('lev_token.csv')

#### Repurchase token if value less than 10c (likely sold in last round)

In [23]:
for token in lev_token.index:
    if lev_token.loc[token]['dollarvalue'] < 0.25:
        rebuy = str(10.001 - lev_token.loc[token]['dollarvalue'])[:6]
        Tclient.create_market_order(token+'-USDT',"buy", funds = rebuy)

#### Create a dataframe for returns on pair combos

In [24]:
ret_frame = pd.DataFrame()
for token in lev_token.index:
    if token[-1] == 'L':
        df = lev_token.loc[lev_token.index.str.startswith(token[:-2])]
        totalvalue = {'token':token[:-2],'currentvalue':df.iloc[0][4] + df.iloc[1][4]}
        ret_frame = ret_frame.append(totalvalue, ignore_index=True)
        ret_frame['purchval'] = (df.iloc[0][3]*df.iloc[0][1])+(df.iloc[1][3]*df.iloc[1][1])
        ret_frame['return%'] = (ret_frame['currentvalue'] - ret_frame['purchval'])*100/ret_frame['purchval']

In [25]:
ret_frame.set_index('token', inplace=True, drop=True)

In [26]:
ret_frame

,currentvalue,purchval,return%
token,,,
FTM,20.251529,20.04022,1.054421
SAND,21.940089,20.04022,9.480278
SUSHI,19.778338,20.04022,-1.306781
NEAR,20.024481,20.04022,-0.078536
MATIC,19.871861,20.04022,-0.840104


In [27]:
for token in ret_frame.index:
    if ret_frame.loc[token]['return%'] >= 25:
        x_long = token +'3L'
        x_short = token +'3S'
        x_long_account = str(lev_token.loc[x_long]['balance']*.99)[:6]
        x_short_account = str(lev_token.loc[x_short]['balance']*.99)[:6]
        
        Tclient.create_market_order(token+'3S-USDT',"sell", size = x_short_account)
        Tclient.create_market_order(token+'3L-USDT',"sell", size = x_long_account)
    else:
        print(f'{token} Not Right Now dude')
        

FTM Not Right Now dude
SAND Not Right Now dude
SUSHI Not Right Now dude
NEAR Not Right Now dude
MATIC Not Right Now dude
